In [29]:
import pandas as pd
import numpy as np


In [2]:
#read in data from ckcherry github
train = pd.read_csv("./full_train_preprocessed_subset.csv",low_memory = False)
test = pd.read_csv("./full_test_preprocessed_subset.csv",low_memory = False)
valid = pd.read_csv("./full_val_preprocessed_subset.csv",low_memory = False)

In [17]:
#check structure of train data
print(train.head())

   Unnamed: 0  annotations                                               text  \
0       11801         1384                    Nobody go to Walmart in sulphur   
1       11802         1384   @emberlhamon @MLidak welcome to the ugly club!!    
2       11803         1384                      getting to help coach jr high   
3       11804         1384  Yo Gotti and Black Youngsta on stage we all th...   
4       11805         1384  @spenceribarra just making sure I haven't been...   

  hashtag                                             tokens  upper_case  \
0     NaN             ['nobody', 'go', 'walmart', 'sulphur']           2   
1     NaN                        ['welcome', 'ugly', 'club']           2   
2     NaN         ['getting', 'help', 'coach', 'jr', 'high']           0   
3     NaN  ['yo', 'gotti', 'black', 'youngsta', 'stage', ...           4   
4     NaN           ['making', 'sure', 'havent', 'replaced']           1   

   num_digits  num_hashtags  num_words  average_word_len

In [4]:
!pip install transformers
!pip install torch

In [5]:
from transformers import BertTokenizer, BertModel
import torch

In [6]:
# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Get the embeddings from the last hidden state (batch_size, sequence_length, embedding_dim)
    embeddings = outputs.last_hidden_state
    # Average over token embeddings to get a single sentence embedding
    sentence_embedding = embeddings.mean(dim=1).squeeze().detach().numpy()
    return sentence_embedding

model.safetensors:  31%|###       | 136M/440M [00:00<?, ?B/s]

In [7]:
# Example tweet
text = "I love machine learning! 😊"
embedding = get_bert_embedding(text)
print(embedding)

[ 8.03204998e-02  1.22918054e-01  1.26589581e-01 -2.66374886e-01
  1.77689239e-01 -2.71641701e-01  3.53137285e-01  4.87102926e-01
 -3.98947775e-01 -1.69440776e-01 -2.50249863e-01 -4.00821030e-01
 -1.73789874e-01  5.81908487e-02 -1.16118103e-01 -1.81075349e-01
  8.15083832e-02  6.11187220e-02  3.20318490e-01  2.56247699e-01
 -9.03702378e-02  2.68987194e-02  2.25703232e-02  3.41205776e-01
  2.90913552e-01  4.92615253e-03  3.70574817e-02  4.45242316e-01
 -6.71408847e-02 -3.03447127e-01  2.84514278e-01  1.84140772e-01
  2.68895179e-01  2.71409571e-01 -1.42630070e-01 -4.19640839e-02
 -1.83588117e-01  1.02859318e-01 -2.58117169e-01  2.05545366e-01
  9.95807499e-02 -4.46917951e-01  1.91553026e-01  2.78909393e-02
  2.06930533e-01 -7.05694199e-01 -3.48678321e-01  1.65436417e-03
  4.39676255e-01 -4.04285043e-02 -7.68633708e-02  4.88362640e-01
  7.35179484e-02 -9.41370577e-02 -2.15701088e-01  5.04661441e-01
 -2.93146610e-01 -4.07490999e-01 -1.45578116e-01 -2.18072414e-01
  1.12465948e-01  1.36503

In [21]:
!pip install tqdm
from tqdm import tqdm

In [27]:
# Create an empty list to store embeddings
embeddings = []

# Generate embeddings based on the 'text' column
print("Generating embeddings for all tweets...")
for text in tqdm(train['text']):  
    embedding = get_bert_embedding(text)
    embeddings.append(embedding)




Generating embeddings for all tweets...


100%|█████████████████████████████████| 133999/133999 [2:08:50<00:00, 17.33it/s]


NameError: name 'np' is not defined

In [45]:
# Convert embeddings list to numpy array for efficient storage
embeddings_array = np.array(embeddings)

# Save the embeddings separately as a numpy file (less memory,faster speed)
np.save('tweet_embeddings.npy', embeddings_array)

# Save a version with just the necessary columns for modeling
modeling_df = train[['text', 'label']].copy()
modeling_df['embedding_file'] = 'tweet_embeddings.npy'  # Reference to embeddings
modeling_df['embedding_index'] = range(len(train))  # Index to retrieve the right embedding
modeling_df.to_csv('train_with_embeddings.csv', index=False)

In [46]:
print(embeddings_array) # Just to check embeddings
print(modeling_df.head()) #Structure of modeling_df

[[ 0.07884582 -0.14852701  0.49020585 ... -0.18483558  0.2448404
   0.34814635]
 [ 0.04497788  0.07456274  0.4310683  ... -0.14134473 -0.15665261
   0.03607357]
 [-0.19066441 -0.2977893  -0.15066284 ...  0.07222332  0.264592
  -0.2900803 ]
 ...
 [ 0.31673098  0.21489394  0.3539819  ... -0.06636077 -0.06858949
   0.0090652 ]
 [ 0.508651    0.03732484 -0.36549538 ... -0.59831285  0.3979687
   0.16410999]
 [ 0.5980494  -0.03892985  0.29503667 ... -0.16583613 -0.02029031
   0.01148175]]
                                                text  label  \
0                    Nobody go to Walmart in sulphur      2   
1   @emberlhamon @MLidak welcome to the ugly club!!       2   
2                      getting to help coach jr high      2   
3  Yo Gotti and Black Youngsta on stage we all th...      2   
4  @spenceribarra just making sure I haven't been...      2   

         embedding_file  embedding_index  
0  tweet_embeddings.npy                0  
1  tweet_embeddings.npy                1  
2  t

In [43]:
from IPython.display import FileLink

# Create a link to download the numpy file with all the embeddings
FileLink('tweet_embeddings.npy')


/Users/nigeleng/Desktop/CS3244/project/tweet_embeddings.npy